# IMPORTS

In [ ]:
import torch
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from torchvision import datasets
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

from data import get_data_transform
from ensembling import Averaging, Voting, Stacking
from model import CNN, ResNet, Inception, VGG, ViT, load_from_exp

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
resnet = load_from_exp(Path("../EXPERIMENTS/resnet_2022-11-27-00-56"))
vgg = load_from_exp(Path("../EXPERIMENTS/vgg_2022-11-27-00-55"))
inception = load_from_exp(Path("../EXPERIMENTS/inception_2022-11-27-00-54"))
bag_of_models = [resnet, vgg, inception]

# DATA

In [ ]:
data_transforms = get_data_transform(image_size=299, data_augmentation=0)

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder("../data/train_images", transform=data_transforms),
    batch_size=64,
    shuffle=False,
)
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder("../data/val_images", transform=data_transforms),
    batch_size=64,
    shuffle=False,
)

## EXTRACT TRAIN AND TEST SET

### TRAIN

In [ ]:
targets = []
for i, (X_train, y_train) in enumerate(train_loader):
    print("Batch", i)
    for j, model in enumerate(bag_of_models):
        if j == 0:
            X_bag_i = model(X_train).detach().numpy()
        else:
            X_bag_i = np.hstack([X_bag_i, model(X_train).detach().numpy()])
    if i == 0:
        X_bag = X_bag_i
    else:
        X_bag = np.vstack([X_bag, X_bag_i])
    targets += y_train.detach().numpy().tolist()

pd.Series(targets).to_csv("y_train.csv")
pd.DataFrame(X_bag).to_csv("X_bag_train.csv")

### TEST

In [ ]:
targets = []
for i, (X_val, y_val) in enumerate(val_loader):
    print("Batch", i)
    for j, model in enumerate(bag_of_models):
        if j == 0:
            X_bag_i = model(X_val).detach().numpy()
        else:
            X_bag_i = np.hstack([X_bag_i, model(X_val).detach().numpy()])
    if i == 0:
        X_bag = X_bag_i
    else:
        X_bag = np.vstack([X_bag, X_bag_i])
    targets += y_val.detach().numpy().tolist()

pd.Series(targets).to_csv("y_val.csv")
pd.DataFrame(X_bag).to_csv("X_bag_val.csv")